In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
import random
import os

# --- 1. CONFIGURATION ---
SEQUENCE_LENGTH = 500  # 10 seconds of data (500 * 0.02s)
INPUT_CHANNELS = 3     # Lateral, Vertical, Longitudinal G-forces
HIDDEN_DIM = 64        # Size of the hidden state in the GRU
NUM_LAYERS = 2         # Number of stacked GRU layers
BATCH_SIZE = 32
LEARNING_RATE = 1e-4

# --- 2. DATA PREPARATION (MOCK DATASET) ---

# NOTE: This section uses a simplified list of 20 coasters with a mock score.
# You will replace the 'load_coaster_data' function to load your actual segmented
# and labeled data where each segment is paired with the coaster's score.

class CoasterScoreDataset(Dataset):
    def __init__(self, data_segments, scores):
        """
        Initializes the dataset. Each segment is paired with the coaster's score.
        :param data_segments: List of NumPy arrays, each (3, SEQUENCE_LENGTH)
        :param scores: List of corresponding coaster scores (0-100)
        """
        self.segments = data_segments
        self.scores = scores

    def __len__(self):
        return len(self.segments)

    def __getitem__(self, idx):
        # Convert NumPy array to Float Tensor
        segment = torch.from_numpy(self.segments[idx]).float()
        # Ensure score is also a Float Tensor
        score = torch.tensor(self.scores[idx]).float()
        return segment, score

# Mock function to load your combined data (replace with actual loading logic)
def load_labeled_data(file_paths_and_scores):
    all_segments = []
    all_scores = []
    
    for file_path, coaster_score in file_paths_and_scores.items():
        # Load the raw data (similar to the SimCLR setup)
        try:
            # We assume a function exists to load and segment one file
            # In your actual project, you would need to associate each segment
            # with its parent coaster's score.
            
            # --- Load data from the specified path ---
            # NOTE: We use os.path.join here for cross-platform compatibility
            
            df = pd.read_csv(file_path, sep=';', header=2, usecols=[1, 2, 3], dtype=np.float32)
            data = df.values.T
            
            # Simple segmentation
            num_timesteps = data.shape[1]
            segments = []
            for start in range(0, num_timesteps - SEQUENCE_LENGTH, SEQUENCE_LENGTH):
                segments.append(data[:, start : start + SEQUENCE_LENGTH])
            
            all_segments.extend(segments)
            all_scores.extend([coaster_score] * len(segments))
            
        except Exception as e:
            print(f"Skipping file {file_path} due to error: {e}")
            continue

    return all_segments, all_scores

# --- 3. THE BIGRU SEQUENCE-TO-VALUE MODEL ---

class BiGRURegressor(nn.Module):
    def __init__(self, input_size=INPUT_CHANNELS, hidden_dim=HIDDEN_DIM, 
                 num_layers=NUM_LAYERS, output_dim=1):
        super(BiGRURegressor, self).__init__()
        
        # 1. BiGRU Encoder
        # batch_first=True means input shape is (Batch, Seq_Len, Channels)
        # However, CNN/RNN inputs usually prefer (Batch, Channels, Seq_Len)
        # We will transpose the input in the forward pass to (Batch, Seq_Len, Channels)
        
        self.gru = nn.GRU(
            input_size=input_size,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True  # Crucial: reads sequence forward and backward
        )
        
        # 2. Dense Regression Head (maps the concatenated hidden states to the score)
        # The output size of a BiGRU is 2 * hidden_dim (one for forward, one for backward)
        self.regressor = nn.Sequential(
            nn.Linear(2 * hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim) # Output a single score value
        )

    def forward(self, x):
        # Input x shape: (B, Channels, Seq_Len) -> e.g., (32, 3, 500)
        
        # Transpose to (B, Seq_Len, Channels) for the GRU
        x = x.transpose(1, 2) # (32, 500, 3) 
        
        # Pass through GRU. We only care about the final hidden state (h_n)
        _, h_n = self.gru(x)
        
        # h_n shape: (2 * Num_Layers, B, Hidden_Dim)
        
        # Concatenate the final hidden states from the last layer (forward and backward)
        # h_n[-2, :, :] is the last forward state
        # h_n[-1, :, :] is the last backward state
        final_forward_backward_state = torch.cat((h_n[-2, :, :], h_n[-1, :, :]), dim=1)
        
        # Pass the concatenated vector through the regression head
        score_prediction = self.regressor(final_forward_backward_state)
        
        # Return prediction (shape: (B, 1))
        return score_prediction.squeeze(1)


# --- 4. EXECUTION AND TRAINING LOOP ---

def main_regression_setup():
    # --- Labeled Data Setup: INPUT SCORES HERE ---
    # This dictionary maps the file path to the coaster's overall user score (0-100).
    # You MUST update this dictionary with the paths and scores for all 20 coasters.
    labeled_coaster_data = {
        # Format: 'folder_name/file_name.csv': Score_Value
        os.path.join('Raw acceleration data', 'SteelVengeance.csv'): 98.5,
        os.path.join('Raw acceleration data', 'Anubis.csv'): 85.2, 
        os.path.join('Raw acceleration data', 'AlpenFury.csv'): 85.2, 
        os.path.join('Raw acceleration data', 'ElToro.csv'): 85.2, 
        os.path.join('Raw acceleration data', 'Hyperia.csv'): 85.2, 
        os.path.join('Raw acceleration data', 'Lightning.csv'): 85.2, 
        os.path.join('Raw acceleration data', 'Pantheon.csv'): 85.2, 
        os.path.join('Raw acceleration data', 'Shambhala.csv'): 85.2, 
        os.path.join('Raw acceleration data', 'Skyrush.csv'): 85.2, 
        os.path.join('Raw acceleration data', 'Taron.csv'): 85.2, 
        os.path.join('Raw acceleration data', 'TwistedCo.csv'): 85.2, 
        os.path.join('Raw acceleration data', 'WickedCyc.csv'): 92.1,
    }
    
    # Load all segments and scores
    all_segments, all_scores = load_labeled_data(labeled_coaster_data)

    if not all_segments:
        print("No valid labeled data loaded. Aborting setup.")
        return

    # Create Dataset and DataLoader
    dataset = CoasterScoreDataset(all_segments, all_scores)
    # Note: Splitting into train/test sets should happen here!
    dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)
    
    print(f"\nTotal labeled segments: {len(all_segments)}")
    print(f"DataLoader initialized with Batch Size: {BATCH_SIZE}")

    # --- Model Training Setup ---
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = BiGRURegressor().to(device)
    
    # Loss function for regression (Mean Squared Error)
    criterion = nn.MSELoss() 
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
    
    print(f"Model initialized on device: {device}")

    # --- Example Training Loop (5 Epochs) ---
    NUM_EPOCHS = 5
    model.train()
    
    for epoch in range(NUM_EPOCHS):
        total_loss = 0
        for segments, scores in dataloader:
            segments = segments.to(device)
            scores = scores.to(device)

            # Forward pass
            predictions = model(segments)
            
            # Calculate Loss
            loss = criterion(predictions, scores)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item() * segments.size(0)

        avg_loss = total_loss / len(dataset)
        print(f"Epoch {epoch+1}/{NUM_EPOCHS}, Loss (MSE): {avg_loss:.4f}")

    print("\n--- BiGRU Regression Training Complete ---")
    
    # Example: How to calculate correlation/R^2 (for the final report)
    # You would do this on a separate test set, not the training set!
    # A low MSE correlates to a strong linear relationship (correlation) between 
    # the features extracted by the BiGRU and the final score.

    # To find Feature Importance (Correlation):
    # 1. Run inference on the test set.
    # 2. Extract the intermediate feature vector (the output of the GRU before the regressor).
    # 3. Use standard statistical methods (e.g., Pearson correlation) to check the correlation 
    #    between these learned features and the final score.

if __name__ == '__main__':
    main_regression_setup()

Skipping file Raw acceleration data\SteelVengeance.csv due to error: Usecols do not match columns, columns expected but not found: [1, 2, 3]
Skipping file Raw acceleration data\Anubis.csv due to error: Usecols do not match columns, columns expected but not found: [1, 2, 3]
No valid labeled data loaded. Aborting setup.


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np
import pandas as pd
import random
import os

# --- Airtime Configuration (Unified: Time and Score) ---
# Coaster: (Start Time (s), End Time (s), Score)
AIRTIME_CONFIGS = {
    "SteelVengeance": (110.0, 203.0, 100.0), 
    "IronGwazi": (74.0, 140.0, 99.8),
    "Zadra": (110.0, 167.0, 99.8),
    "VelociCoas": (49.0, 120.0, 99.8),
    "RidetoHa": (50.0, 107.0, 99.6),
    "Taiga": (70.0, 140.0, 99.5),
    "ArieForce": (80.0, 134.0, 99.4),
    "Maverick": (80.0, 150.0, 99.1),
    "Pantherian": (80.0, 160.0, 99.1),
    "Untamed": (62.0, 125.0, 99.0),
    "Taron": (20.0, 98.0, 98.3),
    "AlpenFury": (35.0, 87.0, 98.3),
    "ElToro": (72.0, 130.0, 98.3),
    "Lightning": (83.0, 140.0, 98.1),
    "TwistedCo": (85.0, 130.0, 98.0),
    "WickedCyc": (82.0, 144.0, 97.8),
    "Shambhala": (77.0, 150.0, 97.7),
    "Skyrush": (70.0, 140.0, 97.7),
    "Hyperia": (49.0, 91.0, 97.6),
    "Pantheon": (80.0, 155.0, 97.4),
    "TopThrillDragster": (7.0, 25.0, 96.1),
    "MillenniumForce": (62.0, 130.0, 95.9),
    "GateKeeper": (66.0, 130.0, 88.2),
    "Raptor": (160.0, 226.0, 87.5),
    "MagnumXL": (80.0, 150.0, 83.4),
    "Valravn": (173.0, 224.0, 82.9),
    "Anubis": (75.0, 120.0, 81.4),
}
TIME_INTERVAL = 0.02
DATA_FOLDER = "accel_data"
# --- End Airtime Configuration ---


# --- 1. CONFIGURATION (REGRESSION) ---
SEQUENCE_LENGTH = 500      
ACCEL_CHANNELS = 3         # Lateral, Vertical, Longitudinal
AIRTIME_FEATURE_COUNT = 4  # Floater, Flojector, Ejector, Total Airtime
TOTAL_RNN_INPUT_SIZE = ACCEL_CHANNELS # Only RNN uses the 3 accel channels
HIDDEN_DIM = 16            
NUM_LAYERS = 2             
BATCH_SIZE = 32
LEARNING_RATE = 5e-4       
TEST_SPLIT_RATIO = 0.2
DROPOUT_RATE = 0.4         

# --- AIRTIME CALCULATION FUNCTION ---

def calculate_airtime(df, start_time, end_time, time_interval=TIME_INTERVAL):
    """
    Calculates the total time spent in Floater, Flojector, and Ejector categories
    for a given coaster dataset within a specified time window.
    """
    if df.empty:
        return np.zeros(AIRTIME_FEATURE_COUNT, dtype=np.float32)

    df_filtered = df[(df['Time'] >= start_time) & (df['Time'] <= end_time)]

    if df_filtered.empty:
        return np.zeros(AIRTIME_FEATURE_COUNT, dtype=np.float32)

    # Floater: -0.25g <= Vertical g <= 0.25g
    floater_mask = (df_filtered['Vertical'] >= -0.25) & (df_filtered['Vertical'] <= 0.25)
    
    # Flojector: -0.75g <= Vertical g < -0.25g
    flojector_mask = (df_filtered['Vertical'] >= -0.75) & (df_filtered['Vertical'] < -0.25)

    # Ejector: Vertical g < -0.75g 
    ejector_mask = (df_filtered['Vertical'] < -0.75)

    # Calculate total time (seconds)
    floater_time = floater_mask.sum() * time_interval
    flojector_time = flojector_mask.sum() * time_interval
    ejector_time = ejector_mask.sum() * time_interval
    total_airtime = floater_time + flojector_time + ejector_time

    # Return as numpy array
    return np.array([floater_time, flojector_time, ejector_time, total_airtime], dtype=np.float32)


# --- 2. DATA PREPARATION ---

class CoasterScoreDataset(Dataset):
    # Dataset now accepts segments (sequence) and airtime (vector) data
    def __init__(self, data_segments, airtime_features, scores):
        self.segments = data_segments
        self.airtime_features = airtime_features
        self.labels = scores.astype(np.float32) 

    def __len__(self):
        return len(self.segments)

    def __getitem__(self, idx):
        segment = torch.from_numpy(self.segments[idx]).float()
        airtime = torch.from_numpy(self.airtime_features[idx]).float()
        label = torch.tensor(self.labels[idx]).float()
        return segment, airtime, label

def load_all_data(config_data):
    all_coaster_data = []
    REQUIRED_COLUMNS = ['Time', 'Lateral', 'Vertical', 'Longitudinal']
    ACCEL_COLUMNS = ['Lateral', 'Vertical', 'Longitudinal']
    FILE_DELIMITER = ',' 
    VERTICAL_G_INDEX = 1 # Index of Vertical in ACCEL_COLUMNS

    # Loop over the new configuration structure: coaster_name maps to (start_time, end_time, score)
    for coaster_name, (start_time, end_time, coaster_score) in config_data.items():
        file_path = os.path.join(DATA_FOLDER, f'{coaster_name}.csv')
        
        try:
            # Load file
            df = pd.read_csv(file_path, sep=FILE_DELIMITER, header=0, dtype=str)
            df = df.iloc[3:].copy() # Assuming similar metadata skip for consistency

            # Clean column names (remove leading/trailing spaces)
            df.columns = df.columns.str.strip()
            
            # Check for required columns and ensure numeric conversion
            missing_cols = [col for col in REQUIRED_COLUMNS if col not in df.columns]
            if missing_cols:
                 # Check for common casing issues if strict match fails
                df.columns = [c if c in REQUIRED_COLUMNS else c.strip().title() for c in df.columns]
                missing_cols = [col for col in REQUIRED_COLUMNS if col not in df.columns]

            if missing_cols:
                raise ValueError(f"Missing required columns in CSV: {missing_cols}")

            # Convert required columns to numeric, coercing errors
            for col in REQUIRED_COLUMNS:
                df[col] = pd.to_numeric(df[col], errors='coerce')
            df = df.dropna(subset=REQUIRED_COLUMNS)

            # --- 1. Calculate Airtime Features (Single vector per coaster) ---
            # Time window is pulled directly from the AIRTIME_CONFIGS tuple
            airtime_features = calculate_airtime(df, start_time, end_time)

            # --- 2. Process Sequence Data (Multiple segments per coaster) ---
            data = df[ACCEL_COLUMNS].values.T # Shape (3, N)
            
            # Normalize input data: subtract 1G from vertical to center around 0
            data[VERTICAL_G_INDEX, :] = data[VERTICAL_G_INDEX, :] - 1.0
            data = data / 5.0  # Scale all by typical max G-force
            
            segments = []
            num_timesteps = data.shape[1]
            for start in range(0, num_timesteps - SEQUENCE_LENGTH + 1, SEQUENCE_LENGTH // 2): # Overlap for better coverage
                segment = data[:, start : start + SEQUENCE_LENGTH]
                if segment.shape[1] == SEQUENCE_LENGTH:
                    segments.append(segment)
            
            if not segments:
                print(f"Skipping file {file_path}: No segments of length {SEQUENCE_LENGTH} could be created.")
                continue

            all_coaster_data.append({
                'segments': segments, 
                'airtime': airtime_features, 
                'score': coaster_score, # Score is now directly from config_data
                'file_path': file_path
            })
            
        except Exception as e:
            print(f"Skipping file {file_path} due to error: {e}")
            continue

    return all_coaster_data

# --- 3. THE MODEL: BiGRU + Airtime Feature Integration ---

class BiGRURegressor(nn.Module):
    def __init__(self, accel_input_size=ACCEL_CHANNELS, airtime_feature_size=AIRTIME_FEATURE_COUNT, 
                 hidden_dim=HIDDEN_DIM, num_layers=NUM_LAYERS, dropout_rate=DROPOUT_RATE):
        super(BiGRURegressor, self).__init__()
        
        # --- RNN for Sequence Data (Acceleration) ---
        self.gru = nn.GRU(
            input_size=accel_input_size,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True,
            dropout=dropout_rate
        )
        
        # --- Linear Layer for Airtime Features ---
        self.airtime_head = nn.Sequential(
            nn.Linear(airtime_feature_size, hidden_dim // 2),
            nn.ReLU(),
            nn.Dropout(dropout_rate / 2)
        )
        
        # Total size after concatenation: (2 * hidden_dim [from BiGRU]) + (hidden_dim // 2 [from airtime_head])
        final_input_size = (2 * hidden_dim) + (hidden_dim // 2)

        # --- Final Regression Head ---
        self.head = nn.Sequential(
            nn.Linear(final_input_size, hidden_dim), 
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim, 1) # Output dim is 1 (the score)
        )

    def forward(self, x_accel, x_airtime):
        
        # 1. Process Acceleration Sequence (x_accel)
        # Input shape: (B, C, L) -> Transpose to (B, L, C) for RNN
        x_accel = x_accel.transpose(1, 2) 
        _, h_n = self.gru(x_accel)
        
        # Concatenate final forward and backward states from all layers
        # h_n shape: (num_layers * 2, B, hidden_dim)
        # Select final forward (h_n[-2]) and backward (h_n[-1]) states
        rnn_embedding = torch.cat((h_n[-2, :, :], h_n[-1, :, :]), dim=1)
        # rnn_embedding shape: (B, 2 * hidden_dim)
        
        # 2. Process Airtime Feature Vector (x_airtime)
        airtime_embedding = self.airtime_head(x_airtime)
        # airtime_embedding shape: (B, hidden_dim // 2)
        
        # 3. Concatenate Embeddings
        combined_embedding = torch.cat((rnn_embedding, airtime_embedding), dim=1)
        # combined_embedding shape: (B, 2*hidden_dim + hidden_dim//2)
        
        # 4. Final Regression
        output = self.head(combined_embedding)
        
        # Scale output to [0, 100] range using sigmoid + scaling
        output_scaled = torch.sigmoid(output) * 100.0
        
        return output_scaled.squeeze(1)

# --- 4. EVALUATION FUNCTION (REGRESSION) ---

def evaluate_model(model, dataloader, device):
    model.eval()
    all_predictions = []
    all_targets = []
    
    with torch.no_grad():
        for segments, airtime_features, labels in dataloader:
            segments = segments.to(device)
            airtime_features = airtime_features.to(device)
            labels = labels.to(device) 

            predictions = model(segments, airtime_features)
            
            all_predictions.extend(predictions.cpu().numpy())
            all_targets.extend(labels.cpu().numpy())

    # Regression Metrics
    r2 = r2_score(all_targets, all_predictions)
    mse = mean_squared_error(all_targets, all_predictions)

    model.train()
    return r2, mse, all_predictions, all_targets

# --- 5. EXECUTION AND TRAINING LOOP ---

def main_regression_setup():
    # --- Load Data from Unified Config ---
    # We now pass AIRTIME_CONFIGS directly as it contains all necessary info (time windows and scores)
    all_coaster_data = load_all_data(AIRTIME_CONFIGS)

    if not all_coaster_data:
        print("No valid labeled data loaded. Aborting setup.")
        return
    
    # We retrieve the file path for the split, which was correctly saved in load_all_data
    coaster_files = [d['file_path'] for d in all_coaster_data]
    
    # Split the coaster files into training and testing lists (Coaster-based split)
    train_coaster_files, test_coaster_files = train_test_split(
        coaster_files, test_size=TEST_SPLIT_RATIO, random_state=42
    )
    
    # Re-aggregate segments based on the split file lists
    train_segments, train_airtime, train_scores = [], [], []
    test_segments, test_airtime, test_scores = [], [], []
    test_coaster_mapping = {}  # Track which segments belong to which coaster

    for coaster_data in all_coaster_data:
        is_train = coaster_data['file_path'] in train_coaster_files
        
        target_segments = train_segments if is_train else test_segments
        target_airtime = train_airtime if is_train else test_airtime
        target_scores = train_scores if is_train else test_scores

        # All segments of a coaster get the same airtime vector and score
        num_segments = len(coaster_data['segments'])
        target_segments.extend(coaster_data['segments'])
        target_airtime.extend([coaster_data['airtime']] * num_segments)
        target_scores.extend([coaster_data['score']] * num_segments)
        
        # Map test segments to their coaster name
        if not is_train:
            coaster_name = os.path.basename(coaster_data['file_path']).replace('.csv', '')
            start_index = len(test_segments) - num_segments
            for i in range(num_segments):
                test_coaster_mapping[start_index + i] = coaster_name
    
    train_scores = np.array(train_scores)
    test_scores = np.array(test_scores)
    train_airtime = np.array(train_airtime)
    test_airtime = np.array(test_airtime)


    train_dataset = CoasterScoreDataset(train_segments, train_airtime, train_scores)
    test_dataset = CoasterScoreDataset(test_segments, test_airtime, test_scores)

    train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
    
    print(f"\nTotal Labeled Segments: {len(train_segments) + len(test_segments)}")
    print(f"Total Coasters: {len(AIRTIME_CONFIGS)}")
    print(f"Training Coasters: {len(train_coaster_files)} | Test Coasters: {len(test_coaster_files)}")
    print(f"Training Segments: {len(train_segments)} | Test Segments: {len(test_segments)}")

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = BiGRURegressor().to(device)
    
    criterion = nn.MSELoss() 
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
    
    NUM_EPOCHS = 50 
    
    print(f"\nStarting training for {NUM_EPOCHS} epochs on {device}...")
    
    for epoch in range(NUM_EPOCHS):
        model.train()
        total_train_loss = 0
        
        for segments, airtime_features, labels in train_dataloader:
            segments = segments.to(device)
            airtime_features = airtime_features.to(device)
            labels = labels.to(device) 

            predictions = model(segments, airtime_features)
            loss = criterion(predictions, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_train_loss += loss.item() * segments.size(0)

        avg_train_loss = total_train_loss / len(train_dataset)

        if (epoch + 1) % 5 == 0 or epoch == NUM_EPOCHS - 1:
            test_r2, test_mse, _, _ = evaluate_model(model, test_dataloader, device)
            print(f"Epoch {epoch+1}/{NUM_EPOCHS} | Train MSE: {avg_train_loss:.4f} | Test MSE: {test_mse:.4f} | Test R2: {test_r2:.4f}")
        else:
            print(f"Epoch {epoch+1}/{NUM_EPOCHS} | Train MSE: {avg_train_loss:.4f}")

    print("\n--- BiGRU Regression Training Complete ---")
    
    final_test_r2, final_test_mse, final_predictions, final_targets = evaluate_model(model, test_dataloader, device)
    
    # Aggregate predictions by coaster
    coaster_scores = {}
    for idx, (pred, target) in enumerate(zip(final_predictions, final_targets)):
        coaster_name = test_coaster_mapping.get(idx, "Unknown")
        if coaster_name not in coaster_scores:
            coaster_scores[coaster_name] = {'predicted': [], 'actual': target}
        coaster_scores[coaster_name]['predicted'].append(pred)
    
    print("\n--- Test Coaster Predictions ---")
    for coaster_name in sorted(coaster_scores.keys()):
        scores_info = coaster_scores[coaster_name]
        avg_pred = np.mean(scores_info['predicted'])
        actual_score = scores_info['actual']
        print(f"{coaster_name}: Predicted Score = {avg_pred:.2f}, Actual Score = {actual_score:.2f}")

if __name__ == '__main__':
    # Fix the random seeds for reproducibility
    torch.manual_seed(42)
    np.random.seed(42)
    random.seed(42)
    main_regression_setup()


Total Labeled Segments: 416
Total Coasters: 12
Training Coasters: 9 | Test Coasters: 3
Training Segments: 299 | Test Segments: 117

Starting training for 50 epochs on cpu...
Epoch 1/50 | Train MSE: 2206.1410
Epoch 2/50 | Train MSE: 1930.7502
Epoch 3/50 | Train MSE: 1599.5743
Epoch 4/50 | Train MSE: 1310.8051
Epoch 5/50 | Train MSE: 1076.0021 | Test MSE: 572.8128 | Test R2: -671.8020
Epoch 6/50 | Train MSE: 781.9806
Epoch 7/50 | Train MSE: 547.3901
Epoch 8/50 | Train MSE: 314.0753
Epoch 9/50 | Train MSE: 174.1340
Epoch 10/50 | Train MSE: 126.9091 | Test MSE: 34.1456 | Test R2: -39.1060
Epoch 11/50 | Train MSE: 89.8590
Epoch 12/50 | Train MSE: 72.1446
Epoch 13/50 | Train MSE: 84.4640
Epoch 14/50 | Train MSE: 50.5933
Epoch 15/50 | Train MSE: 49.4316 | Test MSE: 5.0096 | Test R2: -4.8840
Epoch 16/50 | Train MSE: 40.6479
Epoch 17/50 | Train MSE: 38.3749
Epoch 18/50 | Train MSE: 28.6592
Epoch 19/50 | Train MSE: 26.2116
Epoch 20/50 | Train MSE: 27.2276 | Test MSE: 1.4150 | Test R2: -0.6620
E